# Part I: Understanding the Problem and Data Exploration
<img src="IMG/workflow.png">

## Frame the problem

Before looking at the data it is important to understand how does the company expect to use and benefit from this model? This first brainstorming helps to determine how to frame the problem, what algorithms to select and measure the performance of each one.

## Make Assumptions

It is good to make some assumptions on what possible outcomes we might expect from our analysis according to the available data. Therefore, by knowing the goal we should think which possible factors might affect the sales prediction outcome.

# Start Data Exploration (EDA)
* browse and clean data
* look univariate and multivariate statistics
* isolate and investigate the statistics of groups
* visualize interesting findings
* prepare a small report on your findings (in a seperate notebook)

## Getting Started


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # Ignores any warning
warnings.filterwarnings("ignore")

#Important. for now, we only look at the available train data
train = pd.read_csv("DATA/Train.csv") 
